In [1]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.58.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importar las bibliotecas necesarias
import os
import pandas as pd
from azureml.core import Workspace, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Cargar el espacio de trabajo de Azure
subscription_id = 'bfde7f1d-4598-4e8f-ab3c-31cb49ab70e4'
resource_group = 'desarrollo'
workspace_name = 'amlistea'

# Obtener el espacio de trabajo
workspace = Workspace(subscription_id, resource_group, workspace_name)

# Obtener el dataset registrado por su nombre
dataset_name = 'Dataset'  # Nombre del dataset en el workspace
dataset = Dataset.get_by_name(workspace, name=dataset_name)

# Definir la ruta del archivo donde se descargará el dataset
file_path = './synthetic_customer_data.csv'

# Opción 1: Sobrescribir el archivo si ya existe
# dataset.download(target_path='./', overwrite=True)

# Opción 2: Eliminar el archivo si ya existe antes de descargarlo
if os.path.exists(file_path):
    os.remove(file_path)  # Eliminar el archivo existente

# Descargar el dataset
dataset.download(target_path='./', overwrite=False)

# Leer el archivo CSV descargado
df = pd.read_csv(file_path)

# Vista inicial
print(df.head())

# Eliminación de puntos en columnas numéricas y conversión a float
for col in ['annual_income', 'total_spent', 'avg_purchase_value', 'online_activity_score']:
    df[col] = df[col].astype(str).str.replace('.', '').astype(float)

# Verificar valores nulos en cada columna
null_values = df.isnull().sum()
print("Valores nulos por columna:\n", null_values)

# Contar filas duplicadas
num_duplicates = df.duplicated().sum()
print("Número de filas duplicadas:", num_duplicates)

# Tratamiento de valores nulos
df.fillna({
    'gender': df['gender'].mode()[0],
    'customer_segment': df['customer_segment'].mode()[0],
    'annual_income': df['annual_income'].median(),
    'total_spent': df['total_spent'].median(),
    'avg_purchase_value': df['avg_purchase_value'].median(),
    'online_activity_score': df['online_activity_score'].median()
}, inplace=True)

# Verificar valores nulos nuevamente después de tratamiento
null_values = df.isnull().sum()
print("Valores nulos por columna después del tratamiento:\n", null_values)

# Normalización o estandarización de las columnas numéricas
scaler = StandardScaler()  # Usa MinMaxScaler() para normalizar entre 0 y 1 si prefieres
cols_to_scale = ['annual_income', 'total_spent', 'avg_purchase_value', 'online_activity_score']
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# División en conjunto de entrenamiento y validación
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['customer_segment'])
print("Conjunto de entrenamiento:", train_df.shape)
print("Conjunto de validación:", val_df.shape)

# Preparar características (X) y etiquetas (y) para el conjunto de entrenamiento
X = train_df.drop(columns=['customer_segment', 'customer_id'])  # Eliminar 'customer_id' y la columna de etiqueta
y = train_df['customer_segment']

# Codificación de etiquetas categóricas (customer_segment)
le = LabelEncoder()
y = le.fit_transform(y)  # Convierte 'customer_segment' en valores numéricos (2, 1, 0)

# Convertir otras variables categóricas (gender) en valores numéricos
X = pd.get_dummies(X, columns=['gender'], drop_first=True)

# Definir el modelo
model = RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

# Entrenar el modelo
model.fit(X, y)

# Importar joblib para guardar el modelo
from joblib import dump

# Definir el nombre y la ruta del archivo .joblib
model_path = './random_forest_model.joblib'

# Guardar el modelo
dump(model, model_path)

print(f"Modelo guardado en: {model_path}")

# Preparar características (X_val) y etiquetas (y_val) para el conjunto de validación
X_val = val_df.drop(columns=['customer_segment', 'customer_id'])
X_val = pd.get_dummies(X_val, columns=['gender'], drop_first=True)  # Codificación de 'gender' como en X
y_val = le.transform(val_df['customer_segment'])  # Codificar etiquetas de validación

# Asegurarse de que X_val tiene las mismas columnas que X
missing_cols = set(X.columns) - set(X_val.columns)
for col in missing_cols:
    X_val[col] = 0
X_val = X_val[X.columns]  # Reordenar columnas para que coincidan con X

# Realizar predicciones
y_pred = model.predict(X_val)

# Calcular y mostrar métricas
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

# Calcular MSE, RMSE y MAE
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

{'infer_column_types': 'False', 'activity': 'download.to_path'}
{'infer_column_types': 'False', 'activity': 'download.to_path', 'activityApp': 'FileDataset'}
{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}
  customer_id  age  gender  annual_income   total_spent  num_purchases  \
0   CUST_0001   56    Male          65648  49624.926758             17   
1   CUST_0002   69    Male         123537  46357.348299            196   
2   CUST_0003   46    Male         145991  27043.860801            175   
3   CUST_0004   32  Female          29516  42117.461389             25   
4   CUST_0005   60    Male         132863  26095.803066             94   

   avg_purchase_value  online_activity_score  loyalty_program  \
0          467.729972              52.525842                0   
1          874.799829              11.120875                0   
2          207.419293              86.816930                

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz